In [1]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import metrics
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [2]:
# load excell file
df = pd.read_excel('../data/CollabWriteAnalysisCountCodesLadyorTigerF20nS21S22wGSAnalysis27Jan2023_14Mar2024CleanF.xlsm')
print(df.shape)
df.head()

(611, 33)


,Course,Book ID,Topic,Bookclub,Chat0CREW1,Pseudonym,Message,Message Time,Is Answer,Page,...,BinaryR2Uptake,Pseudonym.1,Message.1,Bookclub.1,R1 Question,R2 Question,R1 Pivot,R2 Pivot,Memo,Old Code Book
0,PIM P1,260,Part 1: What happens next? What is behind the ...,Book Club One,1,pim-01,Hello.,2020-10-20 17:06:00,No,10,...,0,pim-01,Hello.,Book Club One,NaN,NaN,NaN,NaN,NaN,Greeting
1,PIM P1,260,Part 1: What happens next? What is behind the ...,Book Club One,1,pim-01,My assumption is,2020-10-20 17:06:00,No,10,...,0,pim-01,My assumption is,Book Club One,NaN,NaN,NaN,Social/Procedure/UX to Seminar,NaN,Content Discussion
2,PIM P1,260,Part 1: What happens next? What is behind the ...,Book Club One,1,pim-01,that the emphasis on barbarism implies that sh...,2020-10-20 17:06:00,No,10,...,0,pim-01,that the emphasis on barbarism implies that sh...,Book Club One,NaN,NaN,NaN,NaN,NaN,Content Discussion
3,PIM P1,260,Part 1: What happens next? What is behind the ...,Book Club One,1,pim-03,I agree with Cassandra's noticing,2020-10-27 17:58:00,No,10,...,1,pim-03,I agree with Cassandra's noticing,Book Club One,NaN,NaN,NaN,NaN,NaN,Response
4,PIM P1,260,Part 1: What happens next? What is behind the ...,Book Club One,1,pim-03,of the author's word choice of barbarism.,2020-10-27 17:58:00,No,10,...,1,pim-03,of the author's word choice of barbarism.,Book Club One,NaN,NaN,NaN,NaN,NaN,Content Discussion


In [3]:
df.columns

Index(['Course', 'Book ID', 'Topic', 'Bookclub', 'Chat0CREW1', 'Pseudonym',
       'Message', 'Message Time', 'Is Answer', 'Page', 'Response Number',
       'R1 Discussion type', 'R2DiscussionType',
       'R2DiscussionTypeInterpNothers', 'CollapsR2DiscussionTypeInterpNothers',
       'Chat0CREW1B', 'R1 Dialogic spell', 'BinaryR1DialogicSpell',
       'R1 Uptake', 'BinaryR1Uptake', 'R2DialogicSpell',
       'BinaryR2DialogicSpell', 'R2Uptake', 'BinaryR2Uptake', 'Pseudonym.1',
       'Message.1', 'Bookclub.1', 'R1 Question', 'R2 Question', 'R1 Pivot',
       'R2 Pivot', 'Memo', 'Old Code Book'],
      dtype='object')

In [4]:
print('R2DiscussionType', df['R2DiscussionType'].unique(), df['R2DiscussionType'].isna().sum())
print('R2DiscussionTypeInterpNothers', df['R2DiscussionTypeInterpNothers'].unique(), df['R2DiscussionTypeInterpNothers'].isna().sum())
print('CollapsR2DiscussionTypeInterpNothers', df['CollapsR2DiscussionTypeInterpNothers'].unique(), df['CollapsR2DiscussionTypeInterpNothers'].isna().sum())
print('R2DialogicSpell', df['R2DialogicSpell'].unique(), df['R2DialogicSpell'].isna().sum())
print('BinaryR2DialogicSpell', df['BinaryR2DialogicSpell'].unique(), df['BinaryR2DialogicSpell'].isna().sum())
print('R2Uptake', df['R2Uptake'].unique(), df['R2Uptake'].isna().sum())
print('BinaryR2Uptake', df['BinaryR2Uptake'].unique(), df['BinaryR2Uptake'].isna().sum())

R2DiscussionType ['Social' 'Seminar' 'Procedure' 'Other' 'Deliberation' 'UX'
 'Imaginative entry' 'Seminar, Deliberation' 'Social, Deliberation'
 'Deliberation, Seminar' 'Social, Procedure' 'Imaginative'
 'Imaginative Entry'] 0
R2DiscussionTypeInterpNothers ['Social' 'Interpretation' 'Procedure' 'Other' 'Deliberation' 'UX'] 0
CollapsR2DiscussionTypeInterpNothers ['Social/Procedural/UX' 'Interpretation' 'Other' 'Deliberation'] 0
R2DialogicSpell [ 1. nan  2.  3.  4.  5.  6.  7.] 150
BinaryR2DialogicSpell [ 1.  0. nan  2.] 2
R2Uptake [nan 'Affirm' 'Elaborate' 'Filler' 'Clarify' 'Disagree'] 243
BinaryR2Uptake [0 1] 0


In [8]:
names = {1:'R2DiscussionType',2:'R2DiscussionTypeInterpNothers',3:'CollapsR2DiscussionTypeInterpNothers',4:'R2DialogicSpell',5:'BinaryR2DialogicSpell',6:'R2Uptake',7:'BinaryR2Uptake'}
print(df[names[1]].value_counts())

R2DiscussionTypeInterpNothers
Interpretation    335
Deliberation       87
Social             71
UX                 47
Procedure          46
Other              25
Name: count, dtype: int64


In [35]:
df = df.dropna(subset=['Message'])
print(df.shape) # all messages have values
filtered_df1 = df[['Message', 'R2DiscussionType']]
filtered_df2 = df[['Message', 'R2DiscussionTypeInterpNothers']]
filtered_df3 = df[['Message', 'CollapsR2DiscussionTypeInterpNothers']]
filtered_df4 = df[['Message', 'R2DialogicSpell']]
filtered_df5 = df[['Message', 'BinaryR2DialogicSpell']]
filtered_df6 = df[['Message', 'R2Uptake']]
filtered_df7 = df[['Message', 'BinaryR2Uptake']]

# other ways to handle missing labels?
filtered_df4 = filtered_df4.dropna(subset=['R2DialogicSpell'])
filtered_df5 = filtered_df5.dropna(subset=['BinaryR2DialogicSpell'])
filtered_df6 = filtered_df6.dropna(subset=['R2Uptake'])
a=5

(611, 33)


In [38]:
# specify which target to model (filtered1/2/3/.../7)
dataframe = filtered_df3

vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english')
X = vectorizer.fit_transform(dataframe['Message'].values)
y = dataframe.iloc[:, 1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.fit_transform(y_test)

model = xgb.XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Classification report:")
print(metrics.classification_report(y_test, y_pred))

Classification report:
              precision    recall  f1-score   support

           0       0.67      0.35      0.46        23
           1       0.76      0.90      0.83        71
           2       0.00      0.00      0.00         8
           3       0.44      0.57      0.50        21

    accuracy                           0.68       123
   macro avg       0.47      0.46      0.45       123
weighted avg       0.64      0.68      0.65       123



C:\Users\nejcm\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\nejcm\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\nejcm\AppData\Roaming\Python\Python312\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} i

In [39]:
# specify which target to model (filtered1/2/3/.../7)
dataframe = filtered_df3

count_vectorizer = CountVectorizer()
X = count_vectorizer.fit_transform(dataframe['Message'].values)
y = dataframe.iloc[:, 1].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.fit_transform(y_test)

model = xgb.XGBClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

print("Classification report:")
print(metrics.classification_report(y_test, y_pred))

Classification report:
              precision    recall  f1-score   support

           0       0.56      0.43      0.49        23
           1       0.86      0.85      0.85        71
           2       1.00      0.25      0.40         8
           3       0.39      0.62      0.48        21

    accuracy                           0.69       123
   macro avg       0.70      0.54      0.56       123
weighted avg       0.73      0.69      0.69       123

